In [33]:
import os
import pandas as pd
pd.set_option('display.max_columns', 1000)
import numpy as np

# Configure parameter settings
1. Users are required to set path to training "training.data.csv" and testing "testing.data.csv" datasets.\
2. An output dataset named "irae_combined_cleaned.csv" will be saved under the same path after data cleaning. 

In [14]:
# get root directory
print os.getcwd()
ROOT = os.getcwd()

# set path to training and testing datasets
params = {}
params['irae_train'] = os.path.join(ROOT + '/training.data.csv')
params['irae_test'] = os.path.join(ROOT + '/testing.data.csv')

# set path to output dataset
params['output_file_name'] = os.path.join(ROOT + '/irae_combined_cleaned.csv')

SyntaxError: invalid syntax (1573752885.py, line 2)

# Import data
1. Read and merge training and testing datasets.\
2. Add one feature column to label whether the instance belongs to the training or the testing dataset.

In [34]:
# read training dataset
irae_train = pd.read_csv(params['irae_train'], index_col=0)
irae_train['label'] = 'train'

# read testing dataset
irae_test = pd.read_csv(params['irae_test'], index_col=0)
irae_test['label'] = 'test'

# concatenate training and testing datasets
input_data = pd.concat([irae_train, irae_test], join='inner')

# print dataset info
print('Training dataset has ' + str(irae_train.shape[0]) + ' observations and ' + str(irae_train.shape[1]) + ' features')
print('Testing dataset has ' + str(irae_test.shape[0]) + ' observations and ' + str(irae_test.shape[1]) + ' features')
print('Combined dataset has ' + str(input_data.shape[0]) + ' observations and ' + str(input_data.shape[1]) + ' features')

Training dataset has 283 observations and 500 features
Testing dataset has 71 observations and 482 features
Combined dataset has 354 observations and 482 features


In [35]:
# preview dataset
input_data.head()

,SHADOW.ID,Sex,Race,Date.of.diagnosis,Age.at.diagnosis,VitalStatus,Last.Visit,Histology,Immunotherapy.1_DrugName,Immunotherapy.Class,Immunotherapy..1.Start.Date,Immunotherapy..1.End.Date,PD..date..1,Line.of.immunotherapy..1,TKI..1.Drug.Name,TKI..1.Start.date,TKI..1.End.date,PD.Date.TKI..1,Line.of.therapy.TKI..1,TKI..2.Drug.Name,TKI..2.Start.date,TKI..2.End.date,PD.Date.TKI..2,Line.of.therapy.TKI..2,irAE.1,Date.of.irAE.1,Grade.of.irAE.1,irAE.1.category,irAE.2,Date.of.irAE.2,Grade.of.irAE.2,irAE.2.category,irAE.3,Date.of.irAE.3,Grade.of.irAE.3,irAE.3.category,irAE.4,Date.of.irAE.4,Grade.of.irAE.4,irAE.4.category,irAE.5,Date.of.irAE.5,Grade.of.irAE.5,irAE.5.category,PD.L1.Status,PD.L1.Range,Smoking,Packyear,Stage,Report.Date,Molecular.test.performed,ABL1,ABL2,ACVR1B,AKT1,AKT2,AKT3,ALK,APC,AR,ARAF,ARFRP1,ARID1A,ARID1B,ARID2,ASXL1,ATM,ATR,ATRX,AURKA,AURKB,AXIN1,AXL,BAP1,BARD1,BCL2,BCL2L1,BCL2L2,BCL6,BCOR,BCORL1,BCR,BLM,BRAF,BRCA1,BRCA2,BRD4,BRIP1,BTG1,BTK,C11orf30,C17orf39,CARD11,CBFB,CBL,CCND1,CCND2,CCND3,CCNE1,CD274,CD79A,CD79B,CDC73,CDH1,CDK12,CDK4,CDK6,CDK8,CDKN1A,CDKN1B,CDKN2A,CDKN2B,CDKN2C,CEBPA,CHD2,CHD4,CHEK1,CHEK2,CIC,CREBBP,CRKL,CRLF2,CSF1R,CTCF,CTNNA1,CTNNB1,CTOR,CUL3,CYLD,DAXX,DDR2,DH2,DICER1,DNMT3A,DOT1L,EGFR,EGPP,EMSY,EP300,EPHA3,EPHA5,EPHA7,EPHB1,ERBB2,ERBB3,ERBB4,ERG,ERRFI1,ESR1,ETV1,ETV5,ETV6,EWSR1,EZH2,FAM123B,FAM46C,FANCA,FANCC,FANCD2,FANCE,FANCF,FANCG,FANCL,FAS,FAT1,FAT3,FBXW7,FGF10,FGF12,FGF14,FGF19,FGF23,FGF3,FGF4,FGF6,FGFR1,FGFR2,FGFR3,FGFR4,FH,FLCN,FLT1,FLT3,FLT4,FOXL2,FOXP1,FRS2,FUBP1,GABRA6,GATA1,GATA2,GATA3,GATA4,GATA6,GLI1,GNA11,GNA13,GNAQ,GNAS,GPR124,GRIN2A,GRM3,GSK3B,H3F3A,HD2,HGF,HLGGSSCSTC,HNF1A,HRAS,HSD3B1,HSP90AA1,IDH1,IDH2,IGF1R,IGF2,IKBKE,IKZF1,IKZF3,IL7R,INHBA,INPP4B,IRF2,IRF4,IRS2,JAK1,JAK2,JAK3,JUN,KDM5A,KDM5C,KDM6A,KDR,KEAP1,KEL,KF,KIT,KLHL6,KMT2C,KRAS,KZF1,LMO1,LRP1B,LYN,LZTR1,MAGI2,MAP2K1,MAP2K2,MAP2K4,MAP3K1,MCL1,MDM2,MDM4,MED12,MEF2B,MEN1,MER,MET,MITF,MKI67,MLH1,MLL,MLL2,MLL3,MPL,MRE11A,MSH2,MSH3,MSH6,MTOR,MUTYH,MYB,MYC,MYCL1,MYCN,MYD88,MYST3,Microsatellite,NBN,NCOR2,NF1,NF2,NFE2L2,NFKBIA,NKX2.1,NOTCH1,NOTCH2,NOTCH3,NOTCH4,NPM1,NRAS,NSD1,NTRK1,NTRK2,NTRK3,NUP93,PAK3,PAK7,PALB2,PARK2,PARP4,PAX5,PBRM1,PDCD1LG2,PDGFRA,PDGFRB,PDK1,PIK3C2B,PIK3C2G,PIK3CA,PIK3CB,PIK3CG,PIK3R1,PIK3R2,PIM1,PLCG2,PMS2,POLD1,POLE,PPP2R1A,PR124,PRDM1,PREX2,PRKAR1A,PRKCI,PRKDC,PRSS8,PTCH1,PTEN,PTPN11,QKI,RAC1,RAD21,RAD50,RAD51,RAD51L3,RAF1,RANBP2,RARA,RB1,RBM10,RET,RF2,RICTOR,RNF43,ROS1,RPA1,RPTOR,RS2,RUNX1,RUNX1T1,SC2,SDHA,SDHB,SDHC,SDHD,SETBP1,SETD2,SF3B1,SGK1,SLIT2,SMAD2,SMAD3,SMAD4,SMARCA4,SMARCB1,SMO,SNCAIP,SOCS1,SOX10,SOX2,SOX9,SPEN,SPOP,SPTA1,SRC,STAG2,STAT3,STAT4,STK11,SUFU,SYK,TAF1,TBX3,TCF3,TERC,TERT,TET2,TGFBR2,TMPRSS2,TNFAIP3,TNFRSF14,TOP1,TOP2A,TP53,TSC1,TSC2,TSHR,TumorMutationBurden,U2AF1,VEGFA,VHL,WISP3,WT1,XPO1,ZBTB2,ZNF217,ZNF703,TP63,CBLB,ARHGEF12,CLTCL1,EPS15,KAT6A,NR4A3,PICALM,RABEP1,RANBP17,TRIM26,TRIP11,VEGFB,TIPARP,ZN703,MTAP,PIKC3CA,Immunotherapy.2_DrugName,Immunotherapy..2.Start.Date,Immunotherapy..2.End.Date,PD..date..2,Line.of.immunotherapy..2,dataset,TKI..3.Drug.Name,TKI..3.Start.date,TKI..3.End.date,PD.Date.TKI..3,Line.of.therapy.TKI..3,TKI..4.Drug.Name,TKI..4.Start.date,TKI..4.End.date,PD.Date.TKI..4,Line.of.therapy.TKI..4,Immunotherapy.3_DrugName,Immunotherapy..3.Start.Date,Immunotherapy..3.End.Date,PD..date..3,Line.of.immunotherapy..3,I.O.for.irAE.1,I.O.for.irAE.2,I.O.for.irAE.3,I.O.for.irAE.4,I.O.for.irAE.5,I.O.for.irAE.6,irAE.6,Date.of.irAE.6,Grade.of.irAE.6,irAE.6.category,irAE.7,Date.of.irAE.7,Grade.of.irAE.7,irAE.7.category,irAE.8,Date.of.irAE.8,Grade.of.irAE.8,irAE.8.category,ALOX12B,FANCM,PPAEy,RAD51C,SLX4,PDL1_pos_neg,irAE.1_pos_neg,iraAE.1_timing,prior_tki,os_days_update,os_1yr,pfs_days,pfs,pfs_6months,Time.to.irae1,irAE.1_EarlyLate,os_1yr_flag,pfs_6months_flag,label
1,AJ07793T,Female,Unknown,1/24/2018,65,Deceased,9/7/2018,adenocarcinoma,Nivolumab,anti-pd-1,7/27/2018,7/27/2018,9/7/2018,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none reported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Select clinical features of interest
1. Features are selected based on literature and discussion with stakeholders

In [36]:
# select clinical features from input data
emr_features = ['Sex', 'Race', 'Age.at.diagnosis', 'Histology', 'Stage', 'Smoking',
                'Immunotherapy.Class',
                'Immunotherapy..1.Start.Date', 'TKI..1.Start.date', # determine timing of TKI
                'dataset', # this is TKI vs. Non-TKI
                'PD.L1.Status', 'PD.L1.Range', 'PDL1_pos_neg',
                'irAE.1_pos_neg', 'Time.to.irae1', 'irAE.1_EarlyLate', 'irAE.1.category',
                'VitalStatus', 'os_days_update', 'os_1yr_flag', 'pfs', 'pfs_days', 'pfs_6months_flag',
                'label'
                ]

# make selected columns emr_data
emr_data = input_data[emr_features].copy()

In [37]:
# preview emr_data
emr_data.head(2)

,Sex,Race,Age.at.diagnosis,Histology,Stage,Smoking,Immunotherapy.Class,Immunotherapy..1.Start.Date,TKI..1.Start.date,dataset,PD.L1.Status,PD.L1.Range,PDL1_pos_neg,irAE.1_pos_neg,Time.to.irae1,irAE.1_EarlyLate,irAE.1.category,VitalStatus,os_days_update,os_1yr_flag,pfs,pfs_days,pfs_6months_flag,label
1,Female,Unknown,65,adenocarcinoma,IV,non-smoker,anti-pd-1,7/27/2018,NaN,NON-TKI,Not Reported,No Measurement of Negative,NaN,negative,NaN,NaN,NaN,Deceased,42,1,progressed,42.0,1,train
2,Male,White,85,adenocarcinoma,IV,smoker,anti-pd-l1,3/20/2018,NaN,NON-TKI,NaN,No Measurement of Negative,NaN,positive,1.0,Early irAE,IRAE,Deceased,231,1,progressed,97.0,1,train


## Clean existing variables
1. Replace dot (.) & dot dot (..) with underscore (_)\
2. Raname some features for easier interpretation

In [38]:
# replace dot (.) & dot dot (..) with underscore (_)
emr_data.columns = emr_data.columns.str.replace('..', '_', regex=False)
emr_data.columns = emr_data.columns.str.replace('.', '_', regex=False)

# rename some features for easier interpretation
emr_data.rename(columns={'dataset': 'TKI+',
                         'VitalStatus': 'isDeceased',
                         'os_days_update': 'os_days',
                         'pfs': 'isProgressed',
                         'os_1yr_flag': 'os_1yr_isDeceased',
                         'pfs_6months_flag': 'pfs_6months_isProgressed'}, inplace=True)

## Generate new variables
1. Whether a patients had received TKI prior to immunotherapy (Priot_TKI+)\

In [39]:
# filter patients who had TKI prior to immunotherapy
TKI_im_days = pd.to_datetime(emr_data['Immunotherapy_1_Start_Date']) \
            - pd.to_datetime(emr_data['TKI_1_Start_date'])

emr_data['Prior_TKI+'] = 0
emr_data.loc[pd.to_numeric(TKI_im_days) > 0, 'Prior_TKI+'] = 1
emr_data.drop(columns=['Immunotherapy_1_Start_Date', 'TKI_1_Start_date'], inplace=True) # drop dates

## Glance at summary statistics

In [40]:
emr_data.describe(include='all')

,Sex,Race,Age_at_diagnosis,Histology,Stage,Smoking,Immunotherapy_Class,TKI+,PD_L1_Status,PD_L1_Range,PDL1_pos_neg,irAE_1_pos_neg,Time_to_irae1,irAE_1_EarlyLate,irAE_1_category,isDeceased,os_days,os_1yr_isDeceased,isProgressed,pfs_days,pfs_6months_isProgressed,label,Prior_TKI+
count,354,354,354.000000,354,339,351,354,354,327,354,227,354,152.000000,152,152,354,354.000000,354.000000,354,296.000000,354.000000,354,354.000000
unique,2,6,NaN,4,3,2,4,2,30,3,2,2,NaN,2,6,2,NaN,NaN,2,NaN,NaN,2,NaN
top,Male,White,NaN,adenocarcinoma,IV,smoker,anti-pd-1,NON-TKI,Not Reported,No Measurement of Negative,Positive,negative,NaN,Early irAE,IRAE,Deceased,NaN,NaN,progressed,NaN,NaN,train,NaN
freq,190,203,NaN,271,283,246,253,291,100,204,157,202,NaN,76,76,207,NaN,NaN,296,NaN,NaN,283,NaN
mean,NaN,NaN,66.988701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.197368,NaN,NaN,NaN,438.935028,0.511299,NaN,192.091216,0.567797,NaN,0.149718
std,NaN,NaN,10.115474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212.437261,NaN,NaN,NaN,378.946630,0.500580,NaN,232.628565,0.496083,NaN,0.357299
min,NaN,NaN,33.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,6.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000
25%,NaN,NaN,60.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.750000,NaN,NaN,NaN,109.000000,0.000000,NaN,49.000000,0.000000,NaN,0.000000
50%,NaN,NaN,67.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.000000,NaN,NaN,NaN,343.500000,1.000000,NaN,87.500000,1.000000,NaN,0.000000
75%,NaN,NaN,74.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182.000000,NaN,NaN,NaN,660.500000,1.000000,NaN,253.250000,1.000000,NaN,0.000000


# Clean and encode variables
 1. Some features contain instances with low frequency. Group those features into "Other".
 2. Encode categorical variables into dummies

### Sex

In [41]:
print(emr_data.fillna('Unknown').groupby('label')['Sex'].value_counts())

label  Sex   
test   Male       39
       Female     32
train  Male      151
       Female    132
Name: Sex, dtype: int64


In [42]:
# encode sex [Male = 1 / Female = 0]
emr_data['Sex_Male'] = 0
emr_data.loc[emr_data['Sex'] == 'Male', 'Sex_Male'] = 1

print(emr_data.fillna('Unknown').groupby('label')['Sex'].value_counts())

label  Sex   
test   Male       39
       Female     32
train  Male      151
       Female    132
Name: Sex, dtype: int64


### Race

In [43]:
# preview race
print(emr_data.fillna('Unknown').groupby('label')['Race'].value_counts())

label  Race                                  
test   White                                      44
       Unknown                                    15
       Asian                                      10
       American Indian or Alaska Native,White      1
       Black or African American                   1
train  White                                     159
       Asian                                      59
       Unknown                                    52
       Black or African American                  12
       American Indian or Alaska Native            1
Name: Race, dtype: int64


In [44]:
# encode race [group "african american" and "american indian" into "Other" due to low sample size]
emr_data.loc[emr_data['Race'] == 'American Indian or Alaska Native', 'Race'] = 'Other'
emr_data.loc[emr_data['Race'] == 'American Indian or Alaska Native,White', 'Race'] = 'Other'
emr_data.loc[emr_data['Race'] == 'Unknown', 'Race'] = np.NaN

emr_data['Race_White'] = 0
emr_data.loc[emr_data['Race'] == 'White', 'Race_White'] = 1
emr_data['Race_Asian'] = 0
emr_data.loc[emr_data['Race'] == 'Asian', 'Race_Asian'] = 1
emr_data['Race_Black'] = 0
emr_data.loc[emr_data['Race'] == 'Black or African American', 'Race_Black'] = 1

print(emr_data.fillna('Unknown').groupby('label')['Race'].value_counts())

label  Race                     
test   White                         44
       Unknown                       15
       Asian                         10
       Black or African American      1
       Other                          1
train  White                        159
       Asian                         59
       Unknown                       52
       Black or African American     12
       Other                          1
Name: Race, dtype: int64


### Age at diagnosis

In [45]:
# age at diagnosis
emr_data.groupby('label')['Age_at_diagnosis'].describe(include='all')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
test,71.0,67.746479,9.629891,34.0,62.5,68.0,75.5,86.0
train,283.0,66.798587,10.241329,33.0,60.0,67.0,74.0,99.0


### Histology

In [46]:
emr_data.fillna('Unknown').groupby('label')['Histology'].value_counts()

label  Histology     
test   adenocarcinoma     56
       squamous           14
       large cell          1
train  adenocarcinoma    215
       squamous           58
       nos nsclc           7
       large cell          3
Name: Histology, dtype: int64

In [47]:
# encode histology [group "large cell" and "nos nsclc" into "other" due to small sample size]
emr_data.loc[emr_data['Histology'] == 'nos nsclc', 'Histology'] = 'other'
emr_data.loc[emr_data['Histology'] == 'large cell', 'Histology'] = 'other'

emr_data = pd.concat([emr_data, pd.get_dummies(emr_data['Histology'], prefix='Histology', drop_first=False)], axis=1)
emr_data.drop(columns='Histology_other', inplace=True)

emr_data.fillna('Unknown').groupby('label')['Histology'].value_counts()

label  Histology     
test   adenocarcinoma     56
       squamous           14
       other               1
train  adenocarcinoma    215
       squamous           58
       other              10
Name: Histology, dtype: int64

### Stage

In [48]:
emr_data.fillna('Unknown').groupby('label')['Stage'].value_counts()

label  Stage  
test   IV          57
       III         11
       I or II      2
       Unknown      1
train  IV         226
       III         29
       I or II     14
       Unknown     14
Name: Stage, dtype: int64

In [49]:
# encode Stage
emr_data.loc[emr_data['Stage'] == 'Unknown', 'Stage'] = np.NaN

emr_data['Stage'].replace('I or II', 1, inplace=True)
emr_data['Stage'].replace('III', 3, inplace=True)
emr_data['Stage'].replace('IV', 4, inplace=True)

emr_data.fillna('Unknown').groupby('label')['Stage'].value_counts()

label  Stage  
test   4.0         57
       3.0         11
       1.0          2
       Unknown      1
train  4.0        226
       3.0         29
       1.0         14
       Unknown     14
Name: Stage, dtype: int64

### Smoker

In [50]:
emr_data.fillna('Unknown').groupby('label')['Smoking'].value_counts()

label  Smoking   
test   smoker         55
       non-smoker     15
       Unknown         1
train  smoker        191
       non-smoker     90
       Unknown         2
Name: Smoking, dtype: int64

In [51]:
# encode smoking [smoker = 1 and non-smoker = 0]
emr_data.loc[emr_data['Smoking'] == 'Unknown', 'Smoking'] = np.NaN

emr_data['Smoker+'] = 0
emr_data.loc[emr_data['Smoking'] == 'smoker', 'Smoker+'] = 1

emr_data.fillna('Unknown').groupby('label')['Smoking'].value_counts()

label  Smoking   
test   smoker         55
       non-smoker     15
       Unknown         1
train  smoker        191
       non-smoker     90
       Unknown         2
Name: Smoking, dtype: int64

### Immunotherapy class

In [52]:
emr_data.fillna('Unknown').groupby('label')['Immunotherapy_Class'].value_counts()

label  Immunotherapy_Class  
test   anti-pd-1                 52
       anti-pd-l1                19
train  anti-pd-1                201
       anti-pd-l1                79
       anti-pd-l1/ anti-pd-1      2
       anti-pd-1/anti-ctla-4      1
Name: Immunotherapy_Class, dtype: int64

In [53]:
# encode immunotherapy class [dummy variable: anti-pd-1 = 1 and anti-pd-l1 = 0]
emr_data.loc[emr_data['Immunotherapy_Class'] == 'anti-pd-l1/ anti-pd-1', 'Immunotherapy_Class'] = 'other'
emr_data.loc[emr_data['Immunotherapy_Class'] == 'anti-pd-1/anti-ctla-4', 'Immunotherapy_Class'] = 'other'

emr_data['ImClass_anti-pd-l1'] = 0
emr_data.loc[emr_data['Immunotherapy_Class'] == 'anti-pd-l1', 'ImClass_anti-pd-l1'] = 1
emr_data['ImClass_anti-pd-1'] = 0
emr_data.loc[emr_data['Immunotherapy_Class'] == 'anti-pd-1', 'ImClass_anti-pd-1'] = 1

emr_data.fillna('Unknown').groupby('label')['Immunotherapy_Class'].value_counts()

label  Immunotherapy_Class
test   anti-pd-1               52
       anti-pd-l1              19
train  anti-pd-1              201
       anti-pd-l1              79
       other                    3
Name: Immunotherapy_Class, dtype: int64

### TKI

In [54]:
emr_data.fillna('Unknown').groupby('label')['TKI+'].value_counts()

label  TKI+   
test   NON-TKI     60
       TKI         11
train  NON-TKI    231
       TKI         52
Name: TKI+, dtype: int64

In [55]:
# encode TKI+
emr_data['TKI+'].replace('TKI', 1, inplace=True)
emr_data['TKI+'].replace('NON-TKI', 0, inplace=True)

emr_data.fillna('Unknown').groupby('label')['TKI+'].value_counts()

label  TKI+
test   0        60
       1        11
train  0       231
       1        52
Name: TKI+, dtype: int64

### Prior TKI

In [56]:
# encode prior tki
emr_data.fillna('Unknown').groupby('label')['Prior_TKI+'].value_counts()

label  Prior_TKI+
test   0              61
       1              10
train  0             240
       1              43
Name: Prior_TKI+, dtype: int64

### PD-L1 status

In [57]:
emr_data.fillna('Unknown').groupby('label')['PD_L1_Range'].value_counts()

label  PD_L1_Range               
test   No Measurement of Negative     43
       >=50%                          16
       1-49%                          12
train  No Measurement of Negative    161
       >=50%                          78
       1-49%                          44
Name: PD_L1_Range, dtype: int64

In [58]:
# encode PD-L1 status

# there is one patient who had <10% PDL1 status
emr_data.loc[emr_data['PD_L1_Status'] == '<10%', 'PD_L1_Range'] = '1-49%'

# there are six patients who had 0% PD-L1 range but were marked as PD-L1 negative
emr_data.loc[emr_data['PD_L1_Status'] == '0%', 'PDL1_pos_neg'] = 'Negative'

# drop unused feature
emr_data.drop(columns='PD_L1_Status', inplace=True)

# >=50%
emr_data['PD_L1_Range_>=50%'] = 0
emr_data.loc[emr_data['PD_L1_Range'] == '>=50%', 'PD_L1_Range_>=50%'] = 1

# 1-49%
emr_data['PD_L1_Range_1-49%'] = 0
emr_data.loc[emr_data['PD_L1_Range'] == '1-49%', 'PD_L1_Range_1-49%'] = 1

# Negative
emr_data['PD_L1_Range_Negative'] = 0
emr_data.loc[emr_data['PDL1_pos_neg'] == 'Negative', 'PD_L1_Range_Negative'] = 1
emr_data.loc[emr_data['PDL1_pos_neg'] == 'Negative', 'PD_L1_Range'] = 'Negative'

# Positive
emr_data['PD_L1_Range_Positive'] = 0
emr_data.loc[emr_data['PDL1_pos_neg'] == 'Positive', 'PD_L1_Range_Positive'] = 1

# Unknown
emr_data.loc[emr_data['PD_L1_Range'] == 'No Measurement of Negative', 'PD_L1_Range'] = np.NaN

# drop unused feature
emr_data.drop(columns='PDL1_pos_neg', inplace=True)

emr_data.fillna('Unknown').groupby('label')['PD_L1_Range'].value_counts()

label  PD_L1_Range
test   Unknown        31
       >=50%          16
       1-49%          12
       Negative       12
train  Unknown        96
       >=50%          78
       Negative       64
       1-49%          45
Name: PD_L1_Range, dtype: int64

### irAE

In [59]:
emr_data.fillna('Unknown').groupby('label')['irAE_1_pos_neg'].value_counts()

label  irAE_1_pos_neg
test   negative           41
       positive           30
train  negative          161
       positive          122
Name: irAE_1_pos_neg, dtype: int64

In [60]:
# encode irAE [positive = 1 / negative = 0]
emr_data['irAE+'] = 0
emr_data.loc[emr_data['irAE_1_pos_neg'] == 'positive', 'irAE+'] = 1
emr_data.drop(columns='irAE_1_pos_neg', inplace=True)

emr_data.fillna('Unknown').groupby('label')['irAE+'].value_counts()

label  irAE+
test   0         41
       1         30
train  0        161
       1        122
Name: irAE+, dtype: int64

### Time to irAE

In [61]:
# check time to irAE
emr_data.groupby('label')['Time_to_irae1'].describe(include='all')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
test,30.0,96.033333,119.950416,0.0,15.75,52.5,108.0,421.0
train,122.0,169.745902,227.572150,0.0,28.00,70.0,204.5,982.0


In [62]:
# modify variable name
emr_data.rename(columns={'Time_to_irae1': 'Time_to_irAE'}, inplace=True)

### Timing of irAE

In [63]:
emr_data.fillna('Unknown').groupby('label')['irAE_1_EarlyLate'].value_counts()

label  irAE_1_EarlyLate
test   Unknown              41
       Early irAE           17
       Late irAE            13
train  Unknown             161
       Late irAE            63
       Early irAE           59
Name: irAE_1_EarlyLate, dtype: int64

In [64]:
# rename irAE.1_EarlyLate then encode
emr_data.rename(columns={"irAE_1_EarlyLate": "irAE_EarlyLate"}, inplace=True)

emr_data['irAE_early'] = 0
emr_data.loc[emr_data['irAE_EarlyLate'] == 'Early irAE', 'irAE_early'] = 1
emr_data['irAE_late'] = 0
emr_data.loc[emr_data['irAE_EarlyLate'] == 'Late irAE', 'irAE_late'] = 1

emr_data.fillna('Unknown').groupby('label')['irAE_EarlyLate'].value_counts()

label  irAE_EarlyLate
test   Unknown            41
       Early irAE         17
       Late irAE          13
train  Unknown           161
       Late irAE          63
       Early irAE         59
Name: irAE_EarlyLate, dtype: int64

### irAE category

In [65]:
emr_data.fillna('Unknown').groupby('label')['irAE_1_category'].value_counts()

label  irAE_1_category 
test   Unknown              41
       IRAE                 16
       Gastrointestinal      5
       Respiratory           5
       Endocrine             2
       Skin                  2
train  Unknown             161
       IRAE                 60
       Endocrine            20
       Gastrointestinal     15
       Skin                 13
       Respiratory          11
       Hepatobiliary         3
Name: irAE_1_category, dtype: int64

In [66]:
# raname irAE category
emr_data.rename(columns={"irAE_1_category": "irAE_category"}, inplace=True)

emr_data.loc[(emr_data['irAE_category'] != 'IRAE') & (emr_data['irAE+'] > 0), 'irAE_category'] = 'Other'

emr_data = pd.concat([emr_data, pd.get_dummies(emr_data['irAE_category'], prefix='irAE_category', drop_first=False)], axis=1)
emr_data.drop(columns='irAE_category_Other', inplace=True)

emr_data.fillna('Unknown').groupby('label')['irAE_category'].value_counts()

label  irAE_category
test   Unknown           41
       IRAE              16
       Other             14
train  Unknown          161
       Other             62
       IRAE              60
Name: irAE_category, dtype: int64

### OS days

In [67]:
# check OS days
emr_data.groupby('label')['os_days'].describe(include='all')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
test,71.0,366.014085,291.933774,13.0,125.5,266.0,618.5,1024.0
train,283.0,457.229682,396.138104,6.0,105.5,371.0,703.0,1988.0


### PFS days

In [68]:
# check PFS days
emr_data.groupby('label')['pfs_days'].describe(include='all')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
test,66.0,170.515152,182.743304,0.0,55.0,87.0,228.00,874.0
train,230.0,198.282609,245.073829,0.0,47.0,87.5,263.75,1273.0


In [69]:
# if no progression, then pfs = os
emr_data.loc[emr_data['pfs_days'].isnull(), 'pfs_days'] = emr_data.loc[emr_data['pfs_days'].isnull(), 'os_days']

In [70]:
emr_data.groupby('label')['pfs_days'].describe(include='all')

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
test,71.0,199.478873,217.427943,0.0,55.0,105.0,259.5,875.0
train,283.0,305.413428,355.196572,0.0,51.5,127.0,511.0,1742.0


### Vital status

In [71]:
# encode vital status [deceased = 1 / alive = 0]
emr_data['isDeceased'].replace('Deceased', 1, inplace=True)
emr_data['isDeceased'].replace('Alive', 0, inplace=True)

emr_data.fillna('Unknown').groupby('label')['isDeceased'].value_counts()

label  isDeceased
test   1              43
       0              28
train  1             164
       0             119
Name: isDeceased, dtype: int64

### Progression status

In [72]:
# encode pfs status [progressed = 1 / not progressed = 0]
emr_data['isProgressed'].replace('progressed', 1, inplace=True)
emr_data['isProgressed'].replace('no progression', 0, inplace=True)

emr_data.fillna('Unknown').groupby('label')['isProgressed'].value_counts()

label  isProgressed
test   1                66
       0                 5
train  1               230
       0                53
Name: isProgressed, dtype: int64

# 5. Select genetic features of interest

In [73]:
# choose all genomic features
all_genomic_data = input_data.iloc[:, 51:424].astype('object')

# choose selected genomic features
genomic_features = ['EGFR', 'KRAS', 'ALK', 'ROS1', 'BRAF', 'MET', 'RET'] # mutations per NCCN 5.2021 NSCLC guideline

gene_data = all_genomic_data[genomic_features].copy()
gene_data.replace('negative', np.nan, inplace=True)
gene_data.replace('Negative', np.nan, inplace=True)

# binarize mutated genes
# 0: no mutation/ 1: mutation
gene_data = gene_data.notnull().astype('int')

In [74]:
pd.concat([gene_data, input_data['label']], axis=1).groupby('label').sum()

,EGFR,KRAS,ALK,ROS1,BRAF,MET,RET
label,,,,,,,
test,11,18,4,1,0,3,2
train,45,74,13,6,15,18,9


# 6. Save output file

In [75]:
# check output
output_data = pd.concat([emr_data, gene_data], axis=1)
output_data.head(30)

,Sex,Race,Age_at_diagnosis,Histology,Stage,Smoking,Immunotherapy_Class,TKI+,PD_L1_Range,Time_to_irAE,irAE_EarlyLate,irAE_category,isDeceased,os_days,os_1yr_isDeceased,isProgressed,pfs_days,pfs_6months_isProgressed,label,Prior_TKI+,Sex_Male,Race_White,Race_Asian,Race_Black,Histology_adenocarcinoma,Histology_squamous,Smoker+,ImClass_anti-pd-l1,ImClass_anti-pd-1,PD_L1_Range_>=50%,PD_L1_Range_1-49%,PD_L1_Range_Negative,PD_L1_Range_Positive,irAE+,irAE_early,irAE_late,irAE_category_IRAE,EGFR,KRAS,ALK,ROS1,BRAF,MET,RET
1,Female,NaN,65,adenocarcinoma,4.0,non-smoker,anti-pd-1,0,NaN,NaN,NaN,NaN,1,42,1,1,42.0,1,train,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Male,White,85,adenocarcinoma,4.0,smoker,anti-pd-l1,0,NaN,1.0,Early irAE,IRAE,1,231,1,1,97.0,1,train,0,1,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0
3,Male,Asian,61,adenocarcinoma,4.0,smoker,anti-pd-1,0,>=50%,62.0,Early irAE,IRAE,1,642,0,1,168.0,1,train,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,1,0,1,0,1,0,0,0,0,0
4,Female,Black or African American,66,other,4.0,smoker,anti-pd-l1,0,1-49%,NaN,NaN,NaN,0,91,1,1,88.0,1,train,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Male,NaN,63,adenocarcinoma,4.0,smoker,anti-pd-1,0,>=50%,167.0,Late irAE,IRAE,1,328,1,1,328.0,0,train,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0
6,Male,White,67,squamous,3.0,smoker,anti-pd-1,0,Negative,126.0,Late irAE,IRAE,1,179,1,1,179.0,1,train,0,1,1,0,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0
7,Male,White,81,adenocarcinoma,4.0,non-smoker,anti-pd-1,0,>=50%,NaN,NaN,NaN,0,606,0,0,606.0,0,train,0,1,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8,Male,White,48,adenocarcinoma,4.0,smoker,anti-pd-1,0,>=50%,0.0,Early irAE,IRAE,0,1000,0,0,1000.0,0,train,0,1,1,0,0,1,0,1,0,1,1,0,0,1,1,1,0,1,0,1,0,0,0,0,0
9,Female,White,48,adenocarcinoma,4.0,non-smoker,anti-pd-1,1,NaN,NaN,NaN,NaN,1,500,0,1,135.0,1,train,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
12,Female,White,74,adenocarcinoma,1.0,smoker,anti-pd-l1,0,Negative,NaN,NaN,NaN,1,515,0,1,488.0,0,train,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [76]:
# save output file
output_data.to_csv(params['output_file_name'], sep=',', index=False)